### Egg Drop Problem

he following is a description of the instance of this famous puzzle involving n=2 eggs and a building with k=36 floors.

Suppose that we wish to know which stories in a 36-story building are safe to drop eggs from, and which will cause the eggs to break on landing. We make a few assumptions:

* An egg that survives a fall can be used again.
* A broken egg must be discarded.
* The effect of a fall is the same for all eggs.
* If an egg breaks when dropped, then it would break if dropped from a higher floor.
* If an egg survives a fall then it would survive a shorter fall.
* It is not ruled out that the first-floor windows break eggs, nor is it ruled out that the 36th-floor do not cause an egg to break.

If only one egg is available and we wish to be sure of obtaining the right result, the experiment can be carried out in only one way. Drop the egg from the first-floor window; if it survives, drop it from the second floor window. Continue upward until it breaks. In the worst case, this method may require 36 droppings. Suppose 2 eggs are available. What is the least number of egg-droppings that is guaranteed to work in all cases?
The problem is not actually to find the critical floor, but merely to decide floors from which eggs should be dropped so that total number of trials are minimized.

See [The Google example problem](http://www.geeksforgeeks.org/dynamic-programming-set-11-egg-dropping-puzzle/)

See also: [Wiki for Dynamic Programming](https://en.wikipedia.org/wiki/Dynamic_programming)


In [22]:
# %load TestHarness
import inspect
import time
from datetime import timedelta

debugging = False
debugging = True

debugging2 = False

logging = True

def printf(f, *args):
    if len(args): 
        print(f.format(*args))
    else:
        print(f)   

def dbg(f, *args):
    if debugging:
        if len(args):
            printf('  DBG:' + f, *args)
        else:
            print(f)

def dbg2(f, *args):
    if debugging2:
        if len(args):
            printf('  DBG2:' + f, *args)
        else:
            print(f)
        
def log(f, *args):
    if logging:
        printf(f, *args)
        
def log_error(f, *args):
    if logging:
        if len(args):
            printf('*** ERROR:' + f, *args)
        else:
            print('*** ERROR:', f)
        
def class_name(instance):
    return type(instance).__name__

#------------------------------------------------------------------------------
class TestCase(object):
    def __init__(self, name, method, inputs, expected, catchExceptions=False):
        self.name = name
        self.method = method
        self.inputs = inputs
        self.expected = expected
        self.catchExceptions = catchExceptions
        
    def run(self):
        if self.catchExceptions:
            try:
                return self.method(*self.inputs)
            except Exception as x:
                return x
        else:
                return self.method(*self.inputs)

#------------------------------------------------------------------------------
class TestSet(object):
    def __init__(self, cases):
        self.cases = cases
    
    def run_tests(self, repeat=1):
        count = 0
        errors = 0
        total_time = 0
        for case in self.cases:
            count += 1
            start_time = time.time()
            for iteration in range(repeat):
                dbg2("*** Running '{0}' iteration {1}", case.name, iteration+1)
                result = case.run()
            elapsed_time = time.time() - start_time
            total_time += elapsed_time
            if callable(case.expected):
                if not case.expected(result):
                    errors += 1
                    log_error("Test {0} failed. Returned {1}", case.name, result)
            elif result != case.expected:
                errors += 1
                log_error('Test {0} failed. Returned "{1}", expected "{2}"', case.name, result, case.expected)
        if errors:
            log_error("Tests passed: {0}; Failures: {1}", count-errors, errors)
        else:
            log("All {0} tests passed.", count)
        log("Elapsed test time: {0}", timedelta(seconds=total_time))

#------------------------------------------------------------------------------
def elapsed_time(func, *args, msg=''):
    ''' Display the elapsed time of the function.
        Return the function value.
    '''
    stime = time.time()
    result = func(*args)
    etime = time.time() - stime
    log(msg + "Elapsed test time: {0}", timedelta(seconds=etime))
    return result
        


In [141]:
debugging = False

In [144]:
def egg_drop(eggs, floors, memo=None, dynamic=True, info=None, stats=True, depth=0):
    ''' Return the number of tries required to determine the first  floor will
        cause an egg to break if you drop it.
    '''   
    self = egg_drop
    # WOW! frame inspection is PIG SLOW!!!!! Less than 500 calls/second!
    #self = globals()[inspect.getframeinfo(inspect.currentframe()).function]
    if info is None: 
        info={'iterations':0, 'depth':0}
    else:
        info['iterations'] += 1
        info['depth'] = max(info['depth'], depth)
        
    if dynamic:
        key = (eggs, floors)
        if memo is None:
            memo = {}
        else:
            if key in memo:
                return memo[key]
    
    assert eggs > 0 # Can't solve a problem if you can't break an egg.
    if floors == 0: 
        tries = 0 # The answer is already known, since there are no floors.
    elif floors == 1:
        tries = 1 # We have to drop one to see if it breaks.
    elif eggs == 1:
        tries = floors # Need to drop the egg from each floor in turn.
    else:
        tries = float('inf')
        for x in range(1, floors+1):
            egg_broke = self(eggs-1, x-1,      memo, dynamic, info, False, depth+1)
            egg_is_ok = self(eggs,   floors-x, memo, dynamic, info, False, depth+1)
            best_for_floor = max(egg_broke, egg_is_ok)
            tries = min(best_for_floor, tries)
        tries = 1 + tries # This is a trial.

    if dynamic:
        memo[key] = tries
        if stats: dbg('Memo: {0}', [ (k, memo[k]) for k in sorted(memo.keys()) ] )
    if stats: log('Iterations: {0}, Recursive depth: {1}', info['iterations'], info['depth'])
    return tries

In [145]:
def egg_drop_static(eggs, floors):
    return egg_drop(eggs, floors, dynamic=False)

In [146]:
def egg_drop_dynamic(eggs, floors):
    ''' Return the number of tries required to determine the first  floor will
        cause an egg to break if you drop it.
        
        Uses a dynamic programming table to contain all intermediate
        results instead of recursing and using a memo dictionary.
    '''
    key = (eggs, floors)
    table = {}
    
    assert eggs > 0 # Can't solve a problem if you can't break an egg.
    for e in range(1, eggs+1):
        table[(e, 0)] = 0 # The answer is already known, since there are no floors.
        table[(e, 1)] = 1 # # We have to drop one to see if it breaks.
    
    for f in range(1, floors+1):
        table[(1, f)] = f # Need to drop the egg from each floor in turn.
    
    for e in range(2, eggs+1):
        for f in range(2, floors+1):
            k = (e, f)
            table[k] = float('inf')
            for x in range(1, f+1):
                egg_broke = table[(e-1, x-1)]
                egg_is_ok = table[(e,   f-x)]
                best_for_floor = 1 + max(egg_broke, egg_is_ok)
                table[k] = min(best_for_floor, table[k])

    dbg('Table: {0}', [ (k, table[k]) for k in sorted(table.keys()) ] )
    return table[key]

In [147]:
def egg_drop_matrix(eggs, floors):
    ''' Return the number of tries required to determine the first  floor will
        cause an egg to break if you drop it.
        
        Uses a dynamic programming matrix to contain all intermediate
        results instead of recursing and using a memo dictionary.
    '''
    # matrix = [ [0]*(floors+1) ] * (eggs) !!!! WRONG!!!!!  Replicates the rows!!!!
    matrix = [ [0 for cols in range(floors+1)] for rows in range(eggs) ]
        
    assert eggs > 0 # Can't solve a problem if you can't break an egg.
    for e in range(1, eggs+1):
        matrix[e-1][0] = 0 # The answer is already known, since there are no floors.
        matrix[e-1][1] = 1 # # We have to drop one to see if it breaks.
    
    for f in range(1, floors+1):
        matrix[0][f] = f # Need to drop the egg from each floor in turn.
    
    for e in range(2, eggs+1):
        for f in range(2, floors+1):
            matrix[e-1][f] = float('inf')
            for x in range(1, f+1):
                egg_broke = matrix[e-2][x-1]
                egg_is_ok = matrix[e-1][f-x]
                best_for_floor = 1 + max(egg_broke, egg_is_ok)
                assert e > 1
                if best_for_floor < matrix[e-1][f]:
                    matrix[e-1][f] = best_for_floor
                
    dbg('Matrix: {0}', matrix)
    return matrix[eggs-1][floors]

In [148]:
eggs = 2
floors = 100
print(elapsed_time(egg_drop_dynamic, eggs, floors, msg='Number of tries (tabular): '))
print(elapsed_time(egg_drop_matrix,  eggs, floors, msg='Number of tries (matrix) : '))


Number of tries (tabular): Elapsed test time: 0:00:00.009525
14
Number of tries (matrix) : Elapsed test time: 0:00:00.003830
14


In [154]:
eggs = 2
floors = 20
print(elapsed_time(egg_drop,        eggs, floors, msg='Number of tries (dynamic): '))
print(elapsed_time(egg_drop_static, eggs, floors, msg='Number of tries (static) : '))


Iterations: 418, Recursive depth: 19
Number of tries (dynamic): Elapsed test time: 0:00:00.001900
6
Iterations: 1572862, Recursive depth: 19
Number of tries (static) : Elapsed test time: 0:00:03.280220
6


In [155]:
eggs = 5
floors = 1000
print(elapsed_time(egg_drop,         eggs, floors, msg='Number of tries (dynamic): '))
print(elapsed_time(egg_drop_dynamic, eggs, floors, msg='Number of tries (tabular): '))
print(elapsed_time(egg_drop_matrix,  eggs, floors, msg='Number of tries (matrix) : '))


Iterations: 3992000, Recursive depth: 999
Number of tries (dynamic): Elapsed test time: 0:00:08.519198
11
Number of tries (tabular): Elapsed test time: 0:00:03.135530
11
Number of tries (matrix) : Elapsed test time: 0:00:02.177589
11


In [ ]:
import sys
def bar():
     ''' This is a function that learns about itself 
     '''
     felf = globals()[sys._getframe().f_code.co_name]   
     print(felf.__name__, felf.__doc__)
        
     #frame = globals()[sys._getframe().f_code]    
     #print(frame)
     print(bar)

In [ ]:
bar()

In [19]:
def foo():
     ''' This is a function that learns about itself 
     '''
     # WOW! frame inspection is PIG SLOW!!!!! Less than 500 calls/second!
     felf = globals()[inspect.getframeinfo(inspect.currentframe()).function]
     print(felf.__name__, felf.__doc__)
     print(felf)
     print(foo)
     return foo == felf

In [20]:
foo()

foo  This is a function that learns about itself 
     
<function foo at 0x7f5fa0430840>
<function foo at 0x7f5fa0430840>


True

In [55]:
 [ [0]*5 ] * 2

[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]